In [ ]:
import pandas as pd
from pytrends.request import TrendReq

pytrends = TrendReq(hl='es')

from googlesearch import search
import numpy as np

El archivo.txt que se utilice para realizar la búsqueda debe encontrarse en el mismo lugar que este código, y debe estar compuesto por una única columna de palabras sin espacios ni comas, es decir, una palabra abajo de la otra. De lo contrario saldrá un error.

Para abrir el archivo.txt, hay que escribir su nombre en la variable llamada 'texto' de la celda inferior, sin omitir el '.txt'. Todo esto (nombre.txt) tiene que estar entre comillas tal como se muestra en dicha celda.

En caso de querer hacerlo desde Google Colab, para que el código funcione hay que vincular y montar Google drive, de otra forma no se podrá abrir el archivo.txt. En este caso el texto tiene que estar subido a Google Drive. Sin embargo, es recomendable usar Jupyter Notebook debido a que Google Colab impone un límite de búsquedas, generando resultados indeseados.

La variable 'pais' debe ser un código ISO 3166-1 alfa-2. Esto va a determinar el país en el que se va a realizar la búsqueda

La variable 'fecha' es INICIO-FIN. Debe estar en formato YYYY-MM-DD YYYY-MM-DD y entre comillas.

Una vez que se termine de ejecutar todo el código, el archivo .csv va a quedar guardado en el Home Page de Jupyter con el pais y la fecha como nombre. Una vez ahí, se puede descargar y abrir en excel.

In [ ]:
texto = 'Ingredients_2.txt'
pais = 'AR' 
fecha = '2021-1-1 2021-6-30' 

A partir de acá no modificar ninguna celda.

In [ ]:
nombre_del_csv = f'{pais}_{fecha}.csv'
#---------------------------------------------------------------------------------------
ingredients  = open(texto,'r',encoding="UTF-8").read().split('/n')
ingredients = [x for x in ingredients[0].replace("\x00","").split("\n") if bool(x)][1:]
#---------------------------------------------------------------------------------------
desde = fecha[0:9]
print(desde)
hasta = fecha[9:]
print(hasta)

In [ ]:
dictio = {}
try:
    
    for i in ingredients:
        
        print("Comenzando una nueva busqueda...")
        pytrends = TrendReq(timeout=(15, 30))
        pytrends.build_payload(kw_list=[i.strip()], geo=pais, timeframe=fecha)
        main = pytrends.related_queries()
        

        print("guardado en diccionario...")
        dictio[i.strip()] = main
            
except Exception as e:
    print(str(e))

In [ ]:
data = {}
for x in dictio:
    print(x)
    if bool(list(pd.DataFrame(dictio[x][x]["top"]).index.values)):
            data[x] = dictio[x][x]
    else:
        data[x] = { 'top':[] }

In [ ]:
out = {"pais":[],"sabor":[],"query":[],"values":[],"type":[],"desde":[],"hasta":[]}
for sabor in data:
    for key in data[sabor]:
        if bool(list(pd.DataFrame(data[sabor]["top"]).index.values)):
            try:
                for row in range(len(data[sabor][key]["query"])):

                    out["query"].append(data[sabor][key]["query"].values[row])
                    out["values"].append(data[sabor][key]["value"].values[row])
                    out["pais"].append(pais)
                    out["sabor"].append(sabor)
                    out["type"].append(key)
                    out["desde"].append(desde)
                    out["hasta"].append(hasta)
            except TypeError:
                    out["pais"].append(pais)
                    out["sabor"].append(sabor)
                    out["type"].append(key)
                    out["query"].append("-")
                    out["values"].append("-")
                    out["desde"].append(desde)
                    out["hasta"].append(hasta)
        else:
            out["pais"].append(pais)
            out["sabor"].append(sabor)
            out["type"].append(key)
            out["query"].append("-")
            out["values"].append("-")
            out["desde"].append(desde)
            out["hasta"].append(hasta)

In [ ]:
a = pd.DataFrame(out)

In [ ]:
len(set(a['sabor']))

In [ ]:
faltantes = set(ingredients) - set(a['sabor'])

In [ ]:
len(set(a['sabor']))

In [ ]:
pd.DataFrame.from_dict(out).to_csv(nombre_del_csv, index=False)